<b>Author:</b> ...

<b>Contributors:</b> ...


<div class="alert alert-block alert-danger">
Before you start running this notebook, make sure you are using the Hail Genomics Analysis Environment. To do so,
<br/>
    
<ul>
    <li>Click on the <b>cloud analysis environment</b> icon on the righthand side of the screen.</li>
    <li>Inside <b>Recommended environments</b>, select <b>Hail Genomics Analysis</b> which creates a cloud environment for your analyses.</li>
    <li>This analysis can be run with <b>high compute</b> (e.g. 96 CPUs, 624 GB of RAM, 300 workers and 300 preemptibles with 4 CPUs, 15 GB of RAM).</li>
    <li>Click on <b>Next</b>.</li>
</ul>
    
</div>

<h1>Notebook Objectives</h1>

This notebook subsets the short-read v7 VDS to the ~1,027 long-read samples and the ~989 samples for which GATK-SV calls are available.

<b>How to Use this Notebook...</b>

<b>As a tutorial:</b>

...

<b>As a resource:</b>

...

<h2>Relevant Information:</h2>

...

In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
import pysam
from pysam import VariantFile

In [3]:
from google.cloud import storage

In [4]:
import hail as hl
from hail.plot import show
from pprint import pprint

Loading BokehJS ...

## Define helper functions

In [5]:
def mt_exists(gcs_path):
    (gcs_bucket_name, gcs_obj) = re.split("\/", re.sub("gs://", "", gcs_path), maxsplit=1)
    
    storage_client = storage.Client()
    gcs_bucket = storage_client.bucket(gcs_bucket_name)
    stats = storage.Blob(bucket=gcs_bucket, name=f'{gcs_obj}/README.txt').exists(storage_client)
    
    return stats

In [6]:
def vds_exists(gcs_path):
    (gcs_bucket_name, gcs_obj) = re.split("\/", re.sub("gs://", "", gcs_path), maxsplit=1)
    
    storage_client = storage.Client()
    gcs_bucket = storage_client.bucket(gcs_bucket_name)
    stats = storage.Blob(bucket=gcs_bucket, name=f'{gcs_obj}/reference_data/README.txt').exists(storage_client)
    
    return stats

In [7]:
bucket = os.environ['WORKSPACE_BUCKET']
workspace = os.environ['WORKSPACE_NAME']
namespace = os.environ['WORKSPACE_NAMESPACE']

In [5]:
if not os.path.exists("AoU_srWGS_SV_PhaseI.vcf.gz"):
    !gsutil -m cp gs://prod-drc-broad/aou-wgs-sv/phase1/joint-vcf/AoU_srWGS_SV_PhaseI.vcf.gz .
    !gsutil -m cp gs://prod-drc-broad/aou-wgs-sv/phase1/joint-vcf/AoU_srWGS_SV_PhaseI.vcf.gz.tbi .

In [8]:
if not os.path.exists("cohort_AoUSVPhaseII.v7.LRsamples.vcf.gz"):
    !gsutil cp gs://fc-secure-8e5a6fd7-16ae-4796-80ed-8f0463af5ff1/yulia/cohort_AoUSVPhaseII.v7.LRsamples.vcf.gz .

In [9]:
#sr_sv_samples = !zgrep -m1 '^#CHROM' AoU_srWGS_SV_PhaseI.vcf.gz | cut -f10- | sed 's/\t/\n/g'

In [10]:
sr_sv_samples = !zgrep -m1 '^#CHROM' cohort_AoUSVPhaseII.v7.LRsamples.vcf.gz | cut -f10- | sed 's/\t/\n/g'

In [11]:
len(sr_sv_samples)

990

In [12]:
if not os.path.exists("concat_annotated.sens_09.vcf.gz"):
    !gsutil cp gs://fc-secure-fd873afb-038d-44ed-b113-623c141cb95f/releases/sv_integration/GRCh38/v1/concat_annotated.sens_09.vcf.gz .
        
if not os.path.exists("concat_annotated.sens_07.vcf.gz"):        
    !gsutil cp gs://fc-secure-fd873afb-038d-44ed-b113-623c141cb95f/releases/sv_integration/GRCh38/v1/concat_annotated.sens_07.vcf.gz .

In [13]:
sv_sens_09_vcf = 'concat_annotated.sens_09.vcf.gz'
sv_sens_07_vcf = 'concat_annotated.sens_07.vcf.gz'

In [14]:
!cat {sv_sens_09_vcf} | zcat | head -n 2000 | grep -v '^#' | head -n 3 | cut -f1-9

chr1	10147	0	C	CCCTAACCCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCCTAACCCTACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCAACCCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACAACCCTAACCCTAACAACCCTAACAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCTAACCCTAACCCAACCCAACCCTAACCCTAACCCAACCCTAACCCTAACCCTAA	.	.	TRUVARI_ID=chr1-10148-INS-330;SVTYPE=INS;SVLEN=330;GTCNT=1073,0,0,1;F_MISSING=0.999069;NS=1;AN=2;AF=1;MAF=0;AC=2;AC_Het=0;AC_Hom=2;AC_Hemi=0;HWE=1;ExcHet=1	GT:GQ:DR:DV:SCORE:CALIBRATION_SENSITIVITY:SUPP_PBSV:SUPP_SNIFFLES:SUPP_PAV
chr1	10231	1	C	CCCTAACCCTAACCCCTACCCCAACCCCAACCCCAACCCCAACCCCAACCCTTAACCCTAA	.	.	TRUVARI_ID=chr1-10232-INS-60;SVTYPE=INS;SVLEN=60;GTCNT=1073,0,1,0;F_MISSING=0.999069;NS=1;AN=2;AF=0.5;MAF=0.5;AC=1;AC_Het=1;AC_Hom=0;AC_Hemi=0;HWE=1;ExcHet=1	GT:GQ:DR:DV:SCORE:CALIBRATION_SENSITIVITY:SUPP_PBSV:SUPP_SNIFFLES:SUPP_PAV
chr1	10280	2	AACCCTAACCCCAACCCCAACCCCAACCCCAACCCCAACCCCAACCCTAAC	A	.	.	TRUVARI_ID=chr1-10281-DEL-50;SVTY

In [15]:
sv_sens_09_in = VariantFile(sv_sens_09_vcf)  # auto-detect input format

for i, rec in enumerate(sv_sens_09_in):
    print(f'{i} {rec.chrom} {rec.pos} {rec.info.values()}')
    
    if i > 10:
        break

0 chr1 10147 ['chr1-10148-INS-330', 'INS', 330, (1073, 0, 0, 1), (0.9990689754486084,), 1, 2, (1.0,), 0.0, (2,), (0,), (2,), (0,), (1.0,), (1.0,)]
1 chr1 10231 ['chr1-10232-INS-60', 'INS', 60, (1073, 0, 1, 0), (0.9990689754486084,), 1, 2, (0.5,), 0.5, (1,), (1,), (0,), (0,), (1.0,), (1.0,)]
2 chr1 10280 ['chr1-10281-DEL-50', 'DEL', 50, (1074, 0, 0, 0), (1.0,), 0, 0, (None,), None, (0,), (0,), (0,), (0,), (1.0,), (1.0,)]
3 chr1 10300 ['chr1-10301-DEL-103', 'DEL', 103, (1073, 0, 0, 1), (0.9990689754486084,), 1, 2, (1.0,), 0.0, (2,), (0,), (2,), (0,), (1.0,), (1.0,)]
4 chr1 10306 ['chr1-10307-INS-102', 'INS', 102, (1073, 0, 0, 1), (0.9990689754486084,), 1, 2, (1.0,), 0.0, (2,), (0,), (2,), (0,), (1.0,), (1.0,)]
5 chr1 10309 ['chr1-10310-INS-106', 'INS', 106, (1073, 0, 1, 0), (0.9990689754486084,), 1, 2, (0.5,), 0.5, (1,), (1,), (0,), (0,), (1.0,), (1.0,)]
6 chr1 10310 ['chr1-10311-INS-91', 'INS', 91, (1073, 0, 0, 1), (0.9990689754486084,), 1, 2, (1.0,), 0.0, (2,), (0,), (2,), (0,), (1.0,)

[E::idx_find_and_load] Could not retrieve index file for 'concat_annotated.sens_09.vcf.gz'


## List long read samples

In [16]:
lr_sv_samples = !zgrep -m1 '^#CHROM' concat_annotated.sens_09.vcf.gz | cut -f10- | sed 's/\t/\n/g'

In [17]:
len(lr_sv_samples)

1074

In [18]:
!zgrep -m1 '^#CHROM' concat_annotated.sens_09.vcf.gz | cut -f10- | sed 's/\t/\n/g' > samples_1074.txt



gzip: stdout: Broken pipe


In [19]:
with open('samples_1074.txt', 'r') as file:
    sample_names = file.readlines()

sample_names = [name.strip() for name in sample_names]

In [20]:
len(sample_names)

1074

## List long read samples without HPRC samples

In [21]:
common_samples_1027 = [element for element in lr_sv_samples if not (element.startswith('HG') or element.startswith('NA'))]
len(common_samples_1027)

1027

## List long read samples with GATK-SV calls available

In [23]:
#common_samples_989 = list(set(sr_sv_samples) & set(lr_sv_samples))
#len(common_samples_989)

In [24]:
common_samples_990 = list(set(sr_sv_samples) & set(lr_sv_samples))
len(common_samples_990)

990

## Initialize Hail

In [25]:
spark_conf_more_ram = dict()
spark_conf_more_ram["spark.executor.memory"] = "8g"
spark_conf_more_ram["spark.driver.memory"] = "196g"

# hl.init(default_reference='GRCh38', idempotent=True, spark_conf=spark_conf_more_ram)

hl.init(idempotent=True, spark_conf=spark_conf_more_ram)

/opt/conda/lib/python3.10/site-packages/hailtop/aiocloud/aiogoogle/user_config.py:43: UserWarning:

Reading spark-defaults.conf to determine GCS requester pays configuration. This is deprecated. Please use `hailctl config set gcs_requester_pays/project` and `hailctl config set gcs_requester_pays/buckets`.

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.0
SparkUI available at http://saturn-f75e1fa5-6fbc-4dc6-ae19-602e6c4dd082-m.us-central1-c.c.terra-7a376e4e.internal:40299
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.130.post1-c69cd67afb8b
LOGGING: writing to /home/jupyter/AoU_DRC_WGS_LongReads_PacBio/edit/hail-20250328-0502-0.2.130.post1-c69cd67afb8b.log


In [26]:
hl.default_reference('GRCh38')

## Subset v7 VDS to samples that have long reads

In [18]:
vds = hl.vds.read_vds('gs://prod-drc-broad/v7/wgs/with_aian_no_prod/vds/aou_srwgs_short_variants_v7_with_aian_no_prod.vds')

2024-09-06 21:40:23.528 Hail: WARN: You are reading a VDS written with an older version of Hail.
  Hail now supports much faster interval filters on VDS, but you'll need to run either
  `hl.vds.truncate_reference_blocks(vds, ...)` and write a copy (see docs) or patch the
  existing VDS in place with `hl.vds.store_ref_block_max_length(vds_path)`.


In [78]:
if not vds_exists(f'{bucket}/scratch/kvg/srs-subset.1027.chr22.vds'):
    #callset_sample_filtered_1027 = hl.vds.filter_samples(vds, common_samples_1027, keep=True, remove_dead_alleles=True)
    #callset_sample_filtered_1027.write(f'{bucket}/scratch/kvg/srs-subset.1027.chr22.vds', overwrite=True)
    #callset_sample_filtered_1027.write(f'{bucket}/scratch/kvg/srs-subset.1027.vds', overwrite=True)
    
    pass

In [28]:
callset_sample_filtered_1027 = hl.vds.read_vds(f'{bucket}/scratch/kvg/srs-subset.1027.chr22.vds')

2025-03-28 05:04:03.617 Hail: WARN: You are reading a VDS written with an older version of Hail.
  Hail now supports much faster interval filters on VDS, but you'll need to run either
  `hl.vds.truncate_reference_blocks(vds, ...)` and write a copy (see docs) or patch the
  existing VDS in place with `hl.vds.store_ref_block_max_length(vds_path)`.


## Subset v7 VDS to samples that have long reads and GATK-SV calls

In [79]:
if not vds_exists(f'{bucket}/scratch/kvg/srs-subset.989.vds'):
    #callset_sample_filtered_989 = hl.vds.filter_samples(callset_sample_filtered_1027, common_samples_989, keep=True, remove_dead_alleles=True)
    #callset_sample_filtered_989.write(f'{bucket}/scratch/kvg/srs-subset.989.vds', overwrite=True)
    
    pass

In [23]:
callset_sample_filtered_989 = hl.vds.read_vds(f'{bucket}/scratch/kvg/srs-subset.989.vds')

2024-09-06 21:40:59.821 Hail: WARN: You are reading a VDS written with an older version of Hail.
  Hail now supports much faster interval filters on VDS, but you'll need to run either
  `hl.vds.truncate_reference_blocks(vds, ...)` and write a copy (see docs) or patch the
  existing VDS in place with `hl.vds.store_ref_block_max_length(vds_path)`.


In [29]:
if not vds_exists(f'{bucket}/scratch/kvg/srs-subset.990.vds'):
    callset_sample_filtered_990 = hl.vds.filter_samples(callset_sample_filtered_1027, common_samples_990, keep=True, remove_dead_alleles=True)
    callset_sample_filtered_990.write(f'{bucket}/scratch/kvg/srs-subset.990.vds', overwrite=True)

2025-03-28 09:06:01.630 Hail: INFO: wrote matrix table with 2437620004 rows and 990 columns in 84648 partitions to gs://fc-secure-f7d80b48-be60-426f-aa6b-f037a1bf7f34/scratch/kvg/srs-subset.990.vds/reference_data
2025-03-28 09:08:46.425 Hail: INFO: wrote matrix table with 72130558 rows and 990 columns in 84648 partitions to gs://fc-secure-f7d80b48-be60-426f-aa6b-f037a1bf7f34/scratch/kvg/srs-subset.990.vds/variant_data


In [30]:
callset_sample_filtered_990 = hl.vds.read_vds(f'{bucket}/scratch/kvg/srs-subset.990.vds')

2025-03-28 09:08:59.493 Hail: WARN: You are reading a VDS written with an older version of Hail.
  Hail now supports much faster interval filters on VDS, but you'll need to run either
  `hl.vds.truncate_reference_blocks(vds, ...)` and write a copy (see docs) or patch the
  existing VDS in place with `hl.vds.store_ref_block_max_length(vds_path)`.


## Check that we got the number of samples correct

In [31]:
callset_sample_filtered_1027.n_samples()

1027

In [26]:
callset_sample_filtered_989.n_samples()

989

In [32]:
callset_sample_filtered_990.n_samples()

990

## Densify subsetted VDS objects

In [85]:
if not mt_exists(f'{bucket}/scratch/kvg/srs-subset.1027.mt') or True:
    mt_1027 = callset_sample_filtered_1027.variant_data.annotate_entries(
        AD = hl.vds.local_to_global(callset_sample_filtered_1027.variant_data.LAD, 
                                    callset_sample_filtered_1027.variant_data.LA, 
                                    n_alleles = hl.len(callset_sample_filtered_1027.variant_data.alleles), 
                                    fill_value = 0, 
                                    number = 'R')
    )
    
    mt_1027 = mt_1027.annotate_entries(GT = hl.vds.lgt_to_gt(mt_1027.LGT, mt_1027.LA))
    mt_1027 = hl.vds.to_dense_mt(hl.vds.VariantDataset(callset_sample_filtered_1027.reference_data, mt_1027))
    mt_1027 = mt_1027.annotate_rows(info = hl.agg.call_stats(mt_1027.GT, mt_1027.alleles))
    mt_1027.write(f'{bucket}/scratch/kvg/srs-subset.1027.mt', overwrite=True)

2024-09-07 00:28:45.412 Hail: INFO: wrote matrix table with 73161980 rows and 1027 columns in 84648 partitions to gs://fc-secure-f7d80b48-be60-426f-aa6b-f037a1bf7f34/scratch/kvg/srs-subset.1027.mt


In [31]:
mt_1027 = hl.read_matrix_table(f'{bucket}/scratch/kvg/srs-subset.1027.mt')

In [86]:
mt_qc_1027 = hl.sample_qc(mt_1027)

In [88]:
mt_qc_1027.describe()

----------------------------------------
Global fields:
    'tranche_data': array<struct {
        model: str, 
        truth_sensitivity: float64, 
        min_vqslod: float64, 
        filter_name: str
    }>
    'truth_sensitivity_snp_threshold': float64
    'truth_sensitivity_indel_threshold': float64
    'snp_vqslod_threshold': float64
    'indel_vqslod_threshold': float64
----------------------------------------
Column fields:
    's': str
    'sample_qc': struct {
        gq_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        call_rate: float64, 
        n_called: int64, 
        n_not_called: int64, 
        n_filtered: int64, 
        n_hom_ref: int64, 
        n_het: int64, 
        n_hom_var: int64, 
        n_non_ref: int64, 
        n_singleton: int64, 
        n_snp: int64, 
        n_insertion: int64, 
        n_deletion: int64, 
        n_transition: int64, 
        n_transversi

In [87]:
mt_qc_1027.cols().show()

2024-09-07 01:13:24.447 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


+-----------+-------------------------+--------------------------+
| s         | sample_qc.gq_stats.mean | sample_qc.gq_stats.stdev |
+-----------+-------------------------+--------------------------+
| str       |                 float64 |                  float64 |
+-----------+-------------------------+--------------------------+
| "1000151" |                3.85e+01 |                 1.38e+01 |
| "1000513" |                3.79e+01 |                 1.32e+01 |
| "1000920" |                3.85e+01 |                 1.38e+01 |
| "1001399" |                3.88e+01 |                 1.36e+01 |
| "1001980" |                3.83e+01 |                 1.34e+01 |
| "1002322" |                3.80e+01 |                 1.34e+01 |
| "1002826" |                3.87e+01 |                 1.38e+01 |
| "1004266" |                3.84e+01 |                 1.37e+01 |
| "1005038" |                3.87e+01 |                 1.36e+01 |
| "1005444" |                3.84e+01 |                 1.35e+01 |
+-----------+-------------------------+--------------------------+

+------------------------+------------------------+---------------------+
| sample_qc.gq_stats.min | sample_qc.gq_stats.max | sample_qc.call_rate |
+------------------------+------------------------+---------------------+
|                float64 |                float64 |             float64 |
+------------------------+------------------------+---------------------+
|               0.00e+00 |               9.90e+01 |            9.94e-01 |
|               0.00e+00 |               9.90e+01 |            9.94e-01 |
|               0.00e+00 |               9.90e+01 |            9.94e-01 |
|               0.00e+00 |               9.90e+01 |            9.94e-01 |
|               0.00e+00 |               9.90e+01 |            9.94e-01 |
|               0.00e+00 |               9.90e+01 |            9.94e-01 |
|               0.00e+00 |               9.90e+01 |            9.94e-01 |
|               0.00e+00 |               9.90e+01 |            9.94e-01 |
|               0.00e+00 |               9.90e+01 |            9.94e-01 |
|               0.00e+00 |               9.90e+01 |            9.94e-01 |
+------------------------+------------------------+---------------------+

+--------------------+------------------------+----------------------+
| sample_qc.n_called | sample_qc.n_not_called | sample_qc.n_filtered |
+--------------------+------------------------+----------------------+
|              int64 |                  int64 |                int64 |
+--------------------+------------------------+----------------------+
|           72716066 |                      0 |               445914 |
|           72731515 |                      0 |               430465 |
|           72718025 |                      0 |               443955 |
|           72714732 |                      0 |               447248 |
|           72746269 |                      0 |               415711 |
|           72719932 |                      0 |               442048 |
|           72712771 |                      0 |               449209 |
|           72716202 |                      0 |               445778 |
|           72716615 |                      0 |               445365 |
|           72729426 |                      0 |               432554 |
+--------------------+------------------------+----------------------+

+---------------------+-----------------+---------------------+
| sample_qc.n_hom_ref | sample_qc.n_het | sample_qc.n_hom_var |
+---------------------+-----------------+---------------------+
|               int64 |           int64 |               int64 |
+---------------------+-----------------+---------------------+
|            66762013 |         4097921 |             1856132 |
|            66875438 |         4076768 |             1779309 |
|            66785718 |         4111733 |             1820574 |
|            66797836 |         3996124 |             1920772 |
|            67217496 |      

In [84]:
if not mt_exists(f'{bucket}/scratch/kvg/srs-subset.989.mt') or True:
    mt_989 = callset_sample_filtered_989.variant_data.annotate_entries(
        AD = hl.vds.local_to_global(callset_sample_filtered_989.variant_data.LAD, 
                                    callset_sample_filtered_989.variant_data.LA, 
                                    n_alleles = hl.len(callset_sample_filtered_989.variant_data.alleles), 
                                    fill_value = 0, 
                                    number = 'R')
    )
    
    mt_989 = mt_989.annotate_entries(GT = hl.vds.lgt_to_gt(mt_989.LGT, mt_989.LA))
    mt_989 = hl.vds.to_dense_mt(hl.vds.VariantDataset(callset_sample_filtered_989.reference_data, mt_989))
    mt_989 = mt_989.annotate_rows(info = hl.agg.call_stats(mt_989.GT, mt_989.alleles))
    mt_989.write(f'{bucket}/scratch/kvg/srs-subset.989.mt', overwrite=True)

2024-09-06 23:31:19.638 Hail: INFO: wrote matrix table with 72103826 rows and 989 columns in 84648 partitions to gs://fc-secure-f7d80b48-be60-426f-aa6b-f037a1bf7f34/scratch/kvg/srs-subset.989.mt


In [81]:
mt_989 = hl.read_matrix_table(f'{bucket}/scratch/kvg/srs-subset.989.mt')

In [89]:
mt_qc_989 = hl.sample_qc(mt_989)

In [90]:
mt_qc_989.describe()

----------------------------------------
Global fields:
    'tranche_data': array<struct {
        model: str, 
        truth_sensitivity: float64, 
        min_vqslod: float64, 
        filter_name: str
    }>
    'truth_sensitivity_snp_threshold': float64
    'truth_sensitivity_indel_threshold': float64
    'snp_vqslod_threshold': float64
    'indel_vqslod_threshold': float64
----------------------------------------
Column fields:
    's': str
    'sample_qc': struct {
        gq_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        call_rate: float64, 
        n_called: int64, 
        n_not_called: int64, 
        n_filtered: int64, 
        n_hom_ref: int64, 
        n_het: int64, 
        n_hom_var: int64, 
        n_non_ref: int64, 
        n_singleton: int64, 
        n_snp: int64, 
        n_insertion: int64, 
        n_deletion: int64, 
        n_transition: int64, 
        n_transversi

In [93]:
mt_qc_989.cols().show()

FatalError: RemoteException: The directory item limit of /tmp/aggregate_intermediates is exceeded: limit=1048576 items=1048576
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.verifyMaxDirItems(FSDirectory.java:1277)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.addLastINode(FSDirectory.java:1361)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.addINode(FSDirectory.java:1184)
	at org.apache.hadoop.hdfs.server.namenode.FSDirWriteFileOp.addFile(FSDirWriteFileOp.java:579)
	at org.apache.hadoop.hdfs.server.namenode.FSDirWriteFileOp.startFile(FSDirWriteFileOp.java:398)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.startFileInt(FSNamesystem.java:2703)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.startFile(FSNamesystem.java:2596)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.create(NameNodeRpcServer.java:799)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.create(ClientNamenodeProtocolServerSideTransl

FatalError: RemoteException: The directory item limit of /tmp/aggregate_intermediates is exceeded: limit=1048576 items=1048576
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.verifyMaxDirItems(FSDirectory.java:1277)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.addLastINode(FSDirectory.java:1361)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.addINode(FSDirectory.java:1184)
	at org.apache.hadoop.hdfs.server.namenode.FSDirWriteFileOp.addFile(FSDirWriteFileOp.java:579)
	at org.apache.hadoop.hdfs.server.namenode.FSDirWriteFileOp.startFile(FSDirWriteFileOp.java:398)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.startFileInt(FSNamesystem.java:2703)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.startFile(FSNamesystem.java:2596)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.create(NameNodeRpcServer.java:799)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.create(ClientNamenodeProtocolServerSideTransl

In [33]:
if not mt_exists(f'{bucket}/scratch/kvg/srs-subset.990.mt') or True:
    mt_990 = callset_sample_filtered_990.variant_data.annotate_entries(
        AD = hl.vds.local_to_global(callset_sample_filtered_990.variant_data.LAD, 
                                    callset_sample_filtered_990.variant_data.LA, 
                                    n_alleles = hl.len(callset_sample_filtered_990.variant_data.alleles), 
                                    fill_value = 0, 
                                    number = 'R')
    )
    
    mt_990 = mt_990.annotate_entries(GT = hl.vds.lgt_to_gt(mt_990.LGT, mt_990.LA))
    mt_990 = hl.vds.to_dense_mt(hl.vds.VariantDataset(callset_sample_filtered_990.reference_data, mt_990))
    mt_990 = mt_990.annotate_rows(info = hl.agg.call_stats(mt_990.GT, mt_990.alleles))
    mt_990.write(f'{bucket}/scratch/kvg/srs-subset.990.mt', overwrite=True)

2025-03-28 09:16:05.404 Hail: INFO: wrote matrix table with 72130558 rows and 990 columns in 84648 partitions to gs://fc-secure-f7d80b48-be60-426f-aa6b-f037a1bf7f34/scratch/kvg/srs-subset.990.mt


In [34]:
mt_990 = hl.read_matrix_table(f'{bucket}/scratch/kvg/srs-subset.990.mt')

In [35]:
mt_qc_990 = hl.sample_qc(mt_990)

In [36]:
mt_qc_990.describe()

----------------------------------------
Global fields:
    'tranche_data': array<struct {
        model: str, 
        truth_sensitivity: float64, 
        min_vqslod: float64, 
        filter_name: str
    }>
    'truth_sensitivity_snp_threshold': float64
    'truth_sensitivity_indel_threshold': float64
    'snp_vqslod_threshold': float64
    'indel_vqslod_threshold': float64
----------------------------------------
Column fields:
    's': str
    'sample_qc': struct {
        gq_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        call_rate: float64, 
        n_called: int64, 
        n_not_called: int64, 
        n_filtered: int64, 
        n_hom_ref: int64, 
        n_het: int64, 
        n_hom_var: int64, 
        n_non_ref: int64, 
        n_singleton: int64, 
        n_snp: int64, 
        n_insertion: int64, 
        n_deletion: int64, 
        n_transition: int64, 
        n_transversi

In [37]:
mt_qc_990.aggregate_cols(hl.agg.stats(mt_qc_990.sample_qc.r_ti_tv))

2025-03-28 09:16:17.293 Hail: WARN: aggregate_cols(): Aggregates over cols ordered by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


Struct(mean=2.0096745497622273, stdev=0.002684913051356153, min=2.0012562412889787, max=2.0173091551027644, n=990, sum=1989.577804264605)

In [38]:
mt_qc_990.aggregate_cols(hl.agg.stats(mt_qc_990.sample_qc.r_het_hom_var))

Struct(mean=2.183973992295268, stdev=0.10190033852555179, min=1.4475272934819965, max=2.588171391746939, n=990, sum=2162.1342523723156)